In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import xml.etree.ElementTree as ET
from nltk import word_tokenize
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import pymorphy2
%matplotlib inline

C:\Anaconda3\lib\site-packages\gensim-1.0.1-py3.5-win-amd64.egg\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-06-18 23:50:55,839 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
def tokenizator(text):
    sen_clear = []
    for sentence in np.array(text):
        sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
        words = sen_cl.lower().split()
        words_c = []
        for word in words:
            if word != '-':
                words_c.append(morph.parse(word)[0].normal_form)
        sen_clear.append(words_c)
    return sen_clear

In [3]:
def tokenizator_2(sentence):
    sen_cl = re.sub(r'[^\w\d\s\-\n\{\}]', " ", sentence, flags=re.M | re.U | re.I)
    words = sen_cl.lower().split()
    words_c = []
    for word in words:
        if word != '-':
            words_c.append(morph.parse(word)[0].normal_form)
    return words_c

In [4]:
def tree_df(tree_name):
    tree = ET.parse(tree_name)
    sentences = []
    label = []
    data_item = []
    data = []
    polarity=[]
    root = tree.getroot()
    for sentence in root.iter('sentence'):
        for text in sentence.iter('text'):
            data_item.append(text.text)
        for opinion in sentence.iter('Opinion'):
            label.append(opinion.get('category'))
        label = list(set(label))
        data_item.append(label)
        label=[]
        data.append(data_item)
        data_item=[]
    df = pd.DataFrame(data, columns = ['sentence', 'category'])
    return df

In [5]:
name = 'se16_ru_rest_train.xml'
df = tree_df(name)
test_name = 'se16_ru_rest_test.xml'
df_test = tree_df(test_name)

In [6]:
from stop_words import get_stop_words
stop_words = get_stop_words('russian')

In [7]:
mlb = mlb = MultiLabelBinarizer()
df_all = pd.concat([df, df_test], axis = 0)
y_all = mlb.fit_transform(df_all.iloc[:,1])
y_train = y_all[:len(df)]
y_test = y_all[len(df):]

# Word2Vec

In [8]:
morph = pymorphy2.MorphAnalyzer()

2017-06-18 23:50:58,100 : INFO : Loading dictionaries from C:\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-06-18 23:50:59,029 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [9]:
train_sentences = tokenizator(df['sentence'])
test_sentences = tokenizator(df_test['sentence'])

In [10]:
model = gensim.models.Word2Vec(train_sentences,size=1000,window=5,min_count=5, workers=4)

2017-06-18 23:51:18,653 : INFO : collecting all words and their counts
2017-06-18 23:51:18,654 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-18 23:51:18,668 : INFO : collected 4255 word types from a corpus of 41494 raw words and 3655 sentences
2017-06-18 23:51:18,669 : INFO : Loading a fresh vocabulary
2017-06-18 23:51:18,677 : INFO : min_count=5 retains 1047 unique words (24% of original 4255, drops 3208)
2017-06-18 23:51:18,678 : INFO : min_count=5 leaves 36567 word corpus (88% of original 41494, drops 4927)
2017-06-18 23:51:18,686 : INFO : deleting the raw counts dictionary of 4255 items
2017-06-18 23:51:18,688 : INFO : sample=0.001 downsamples 69 most-common words
2017-06-18 23:51:18,689 : INFO : downsampling leaves estimated 27012 word corpus (73.9% of prior 36567)
2017-06-18 23:51:18,690 : INFO : estimated required memory for 1047 words and 1000 dimensions: 8899500 bytes
2017-06-18 23:51:18,696 : INFO : resetting layer weights
2017-06-18 23:51

In [11]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,))
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords != 0.0:
        featureVec = np.true_divide(featureVec,nwords)
    else:
        featureVec = np.zeros((num_features,))
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features))
    #reviewFeatureVecs = np.array((len(reviews),num_features))
    # 
    # Loop through the reviews
    for i in range(len(reviews)):
        reviewFeatureVecs[i] = makeFeatureVec(reviews[i], model, num_features)
        
    #for review in reviews:

       # Call the function (defined above) that makes average feature vectors
       #reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
       #    num_features)
       #
       # Increment the counter
       #counter = counter + 1.
    
    return reviewFeatureVecs

In [12]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.


trainDataVecs = getAvgFeatureVecs(train_sentences, model, 1000 )

print ("Creating average feature vecs for test reviews")

testDataVecs = getAvgFeatureVecs(test_sentences, model, 1000 )

Creating average feature vecs for test reviews


In [33]:
def classify_random_forest(X_train, y_train, X_test, y_test):
    clf = OneVsRestClassifier(RandomForestClassifier( n_estimators = 100 ))
    clf.fit(X_train,y_train)
    clf.predict_proba(X_test)
    classes = list(mlb.classes_)
    classes_final = []
    for line in clf.predict_proba(X_test):
        cl = []
        for i in range(len(classes)):
            if line[i] >= 0.2:
                cl.append(classes[i])
        classes_final.append(cl)
    classes_final_s = pd.Series(classes_final)
    df_test_f = pd.concat([df_test,classes_final_s], axis = 1)
    df_test_f.rename(columns={0:"predicted classes"})
    #accuracy=clf.score(X_test,y_test)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average = 'macro')
    recall = metrics.recall_score(y_test, y_pred, average = 'macro')
    f1=metrics.f1_score(y_test, y_pred, average = 'macro')
    return df_test_f, accuracy, precision, recall, f1, clf.predict_proba(X_test)


In [34]:
df_test_f_w2v_SVC, accuracy_w2v_SVC, precision_w2v_SVC, recall_w2v_SVC, f1_w2v_SVC, probabilities = classify_SVC(trainDataVecs,y_train,testDataVecs,y_test)
print('accuracy = {:0.5f}, precision = {:0.5f}, recall = {:0.5f}, f1 = {:0.5f}'.format(accuracy_w2v_SVC, precision_w2v_SVC, recall_w2v_SVC, f1_w2v_SVC))

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


accuracy = 0.24897, precision = 0.00000, recall = 0.00000, f1 = 0.00000


In [35]:
classes = list(mlb.classes_)
print(classes)

['AMBIENCE#GENERAL', 'DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE_OPTIONS', 'FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#PRICES', 'SERVICE#GENERAL']


In [36]:
classes_final=[]
for line in probabilities:
    cl = []
    for i in range(len(classes)):
        if line[i] >= 0.2:
            cl.append(classes[i])
    classes_final.append(cl)
    cl=[]
print(classes_final)
y_pred = mlb.transform(classes_final) 

[['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['SERVICE#GENERAL'], ['FOOD#QUALITY', 'FOOD#STYLE_OPTIONS'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'SERVICE#GENERAL'], ['FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'RESTAURANT#GENERAL'], ['AMBIENCE#GENERAL', 'RESTAURANT#GENERAL'], ['FOOD#QUALITY', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['RESTAURANT#GENERAL'], ['FOOD#QUALITY', 'RESTAURANT#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'SERVICE#GENERAL'], ['FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['AMBIENCE#GENERAL', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['FOOD#QUALITY', 'FOOD#STYLE_OPTIONS', 'RESTAURANT#GENERAL', 'SERVICE#GENERAL'], ['RESTAURANT#GENERAL'], ['RESTAURANT#GENERAL', 'SERVICE#GEN

In [37]:
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred, average = 'macro')
recall = metrics.recall_score(y_test,y_pred, average = 'macro')
f1=metrics.f1_score(y_test, y_pred, average = 'macro')

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
accuracy

0.032258064516129031

In [39]:
precision

0.19464146019288867

In [40]:
recall

0.40374770175249908

In [41]:
f1

0.17448511523240562